In [22]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler 
import numpy as np

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')

train


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [23]:
# PassengerId : 각 승객별 고유번호
# Survived : 생존 여부 (종속 변수)
# Pclass : 1등석, 2등석, 3등석
# Name : 이름
# Gender : 성별
# Age : 나이
# SibSp : 동반한 Sibling(형제자매)와 Spouse(배우자)의 수
# Parch : 동반한 Parent(부모) Child(자식)의 수
# Ticket : 티켓의 고유번호
# Fare : 티켓의 요금
# Cabin : 객실 번호
# Embarked : 탑승한 항구(지역)
# C = Cherbourg(셰르부르)
# Q = Queenstown(퀸즈타운)
# S = Southampton(사우샘푸턴)

In [24]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [27]:
train.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

# 전처리

In [ ]:
# Name 삭제
train = train.drop(['Name'],axis=1)
test = test.drop(['Name'],axis=1)

In [ ]:
# 결측치
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
# 결측치 처리
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [ ]:
# Embarked 'S'가 다수이므로 'S'로 채워주기
train["Embarked"].fillna('S', inplace=True)
test["Embarked"].fillna('S', inplace=True)
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# sex -> 0,1로 변경
train.loc[train['Sex'] == 'male', 'Sex'] = 0
train.loc[train['Sex'] == 'female', 'Sex'] = 1

# sex, int 형으로 변환
train['Sex'] = train['Sex'].astype('int64')

# # Embarked 0,1,2로 변경
train.loc[train['Embarked'] == 'S','Embarked'] = 0
train.loc[train['Embarked'] == 'C','Embarked'] = 1 
train.loc[train['Embarked'] == 'Q','Embarked'] = 2 

# int형으로 변환
train['Embarked'] = train['Embarked'].astype('int64')

# age NaN값 평균으로 넣어주기
train['Age'].fillna(train['Age'].mean(),inplace=True)
train['Age'] = train['Age'].astype('int64')

train = train.drop(['Ticket','Cabin','PassengerId'],axis=1)
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22,1,0,7.2500,0
1,1,1,1,38,1,0,71.2833,1
2,1,3,1,26,0,0,7.9250,0
3,1,1,1,35,1,0,53.1000,0
4,0,3,0,35,0,0,8.0500,0
...,...,...,...,...,...,...,...,...
886,0,2,0,27,0,0,13.0000,0
887,1,1,1,19,0,0,30.0000,0
888,0,3,1,29,1,2,23.4500,0
889,1,1,0,26,0,0,30.0000,1


In [ ]:
# sex -> 0,1로 변경
test.loc[test['Sex'] == 'male', 'Sex'] = 0
test.loc[test['Sex'] == 'female', 'Sex'] = 1

# sex, int 형으로 변환
test['Sex'] = test['Sex'].astype('int64')

# # Embarked 0,1,2로 변경
test.loc[test['Embarked'] == 'S','Embarked'] = 0
test.loc[test['Embarked'] == 'C','Embarked'] = 1 
test.loc[test['Embarked'] == 'Q','Embarked'] = 2 

# int형으로 변환
test['Embarked'] = test['Embarked'].astype('int64')

# age NaN값 평균으로 넣어주기
test['Age'].fillna(test['Age'].mean(),inplace=True)
test['Age'] = test['Age'].astype('int64')

test = test.drop(['Ticket','Cabin','PassengerId'],axis=1)
test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34,0,0,7.8292,2
1,3,1,47,1,0,7.0000,0
2,2,0,62,0,0,9.6875,2
3,3,0,27,0,0,8.6625,0
4,3,1,22,1,1,12.2875,0
...,...,...,...,...,...,...,...
413,3,0,30,0,0,8.0500,0
414,1,1,39,0,0,108.9000,1
415,3,0,38,0,0,7.2500,0
416,3,0,30,0,0,8.0500,0


In [ ]:

# Fare 결측치
test['Fare'].fillna(test['Fare'].mean(),inplace=True)
test.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [ ]:
train_x = train.drop('Survived',axis=1)
train_y = train[['Survived']]
test_x = test
print(train_x.shape,train_y.shape,test_x.shape)
train_x

(891, 7) (891, 1) (418, 7)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22,1,0,7.2500,0
1,1,1,38,1,0,71.2833,1
2,3,1,26,0,0,7.9250,0
3,1,1,35,1,0,53.1000,0
4,3,0,35,0,0,8.0500,0
...,...,...,...,...,...,...,...
886,2,0,27,0,0,13.0000,0
887,1,1,19,0,0,30.0000,0
888,3,1,29,1,2,23.4500,0
889,1,0,26,0,0,30.0000,1


In [ ]:
# 랜덤포레스트
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(train_x,train_y)
Y_pred = random_forest.predict(test_x)
random_forest.score(train_x,train_y)

C:\Users\jangmark\AppData\Local\Temp\ipykernel_4268\4127983320.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest.fit(train_x,train_y)


0.9764309764309764

In [ ]:
# 최근접이웃
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(train_x, train_y)
Y_pred = knn.predict(test_x)
knn.score(train_x, train_y)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode

0.835016835016835

In [ ]:
# SVC
svc = SVC()
svc.fit(train_x, train_y)
Y_pred = svc.predict(test_x)
svc.score(train_x, train_y)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6812570145903479

In [ ]:
# 교차 검증

# k-fold 객체 생성
kf = KFold(n_splits=5, shuffle=True, random_state=55)

# 모델생성 - 랜덤포레스트
Rf = RandomForestClassifier()

# k-Fold 교차 검증 수행
predictions = np.zeros((test_x.shape,1))
for train_idx, test_idx in kf.split(train_x):
    X_train, y_train = train_x.iloc[train_idx], train_y.iloc[train_idx]
    X_test, y_test = train_x.iloc[test_idx], train_y.iloc[test_idx]
    Rf.fit(X_train,y_train) # 교차검증 후 모델에 적용

    predictions = Rf.predict_proba(test_x)

Rf.score(train_x,train_y)

In [ ]:
# 로지스틱 회귀

lr = LogisticRegression()
lr.fit(train_x,train_y)
lr.score(train_x,train_y)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8024691358024691

In [ ]:
# 결정트리
Dt = DecisionTreeClassifier()
Dt.fit(train_x,train_y)
Dt.score(train_x,train_y)
Dt.predict_proba(test_x)

array([[1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.

In [ ]:
# 정규화 
ss = StandardScaler()
ss.fit(train_x)
scaled_x = ss.transform(train_x)
test_scaled = ss.transform(test_x)

lr = LogisticRegression()
lr.fit(scaled_x,train_y)
lr.score(scaled_x,train_y)



c:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8002244668911336